--- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [1]:
example_schematic_str = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""

In [2]:
import re



def process_scheme_line(scheme_line_str: str) -> tuple[set[int], list[re.Match]]:
    """
    Returns a tuple of list of Matches for symbols and list of Matches for numbers
    """
    special_matches = set()
    for char_idx, char_val in enumerate(scheme_line_str):
        if not (char_val.isnumeric() or char_val == "."):
            special_matches.add(char_idx)
    
    special_matches_plus_adjacent = set()
    for smi in special_matches:
        min_trunc = max(0, smi - 1)
        max_trunc = min(len(scheme_line_str) - 1, smi + 1)
        special_matches_plus_adjacent = special_matches_plus_adjacent.union({min_trunc, smi, max_trunc})
    digits_matches = list(re.finditer(r"(\d+)", scheme_line_str))
    return special_matches_plus_adjacent, digits_matches


def find_digits_adjacent_from_trio_lines(line_trio: list[tuple[set[int], list[re.Match]]]):
    indices_included = set()
    output_sum = 0
    for special_indices, _ in line_trio:
        indices_included = indices_included.union(special_indices)

    for digit_matches in line_trio[1][1]:
        if set(range(*digit_matches.span())).intersection(set(indices_included)):
            output_sum += int(digit_matches.group(1))
    return output_sum



def process_engine_scheme_string(scheme_str: str) -> int:
    scheme_lines_list = scheme_str.strip().split("\n")
    total_sum = 0
    process_line_trio = [
        process_scheme_line(""),
        process_scheme_line(scheme_lines_list[0]),
        process_scheme_line(scheme_lines_list[1]),
    ]
    total_sum += find_digits_adjacent_from_trio_lines(process_line_trio)
    for scheme_line in scheme_lines_list[2:]:
        process_line_trio.pop(0)
        process_line_trio.append(process_scheme_line(scheme_line))
        total_sum += find_digits_adjacent_from_trio_lines(process_line_trio)

    # process last line
    process_line_trio.pop(0)
    process_line_trio.append(process_scheme_line(""))
    total_sum += find_digits_adjacent_from_trio_lines(process_line_trio)

    return total_sum




In [3]:
assert process_engine_scheme_string(example_schematic_str) == 4361

In [4]:
with open("../inputs/day3_input.txt") as f:
    input_str = f.read()


In [5]:
process_engine_scheme_string(input_str)

512794